# 基于模式匹配的对话机器人实现

## 英文版

In [ ]:
def is_variable(pat):
    '''
    判断是不是 '?X'形式
    '''
    if pat.startswith('?') and (not pat.startswith('?*')):
        return True
    return False
    
def is_variable_star(pat):
    '''
    判断是不是 '?*X'形式
    '''
    if pat.startswith('?*'):
        return True
    return False

In [ ]:
def segment_match(saying, key):
    '''
    处理'?*X'形式。 即[('?*A',['an', 'iphone'])]
    '''
    pat, rest = key[0], key[1:]
    #print("pat",pat)
    #pat.replace('?*','?')
    a = pat[2]
    pat = '?'+ a
    #print("pat",pat)
    if not rest:
        return (pat,saying), len(saying)
    for index, token in enumerate(saying):
        if is_match(rest, saying[index:]):
            return (pat, saying[:index]), index
        
    return (pat,saying), len(saying)

In [ ]:
def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])


In [ ]:
def pat_to_dict(pattern):
    return {k:' '.join(v) if isinstance(v, list) else v for k,v in pattern}

def subsitite(answer, pattern):
    if not answer:
        return []
    return [pattern.get(answer[0], answer[0])] + subsitite(answer[1:], pattern)


In [ ]:
def pat_match(saying, key):
    '''
    得到各个占位符真实的值 。占位符包括 '?X','?*X'
    input:
        saying: 比如：['I','need','iPhone']
        key: 比如: ['I','need','?X']
    return:
        [('?x', 'hello'), ('?y','good')] 或者 None
    '''
    if (not saying) or (not key):
        return []
    pat = key[0]
    #判断是不是 '?X'形式
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match(saying[1:], key[1:])
    
    #判断是不是 '?*X'形式
    if is_variable_star(pat):
        (pat, value), index = segment_match(saying, key)
        return [(pat, value)] + pat_match(saying[1:], key[index:])
    
    if pat == saying[0]:
        return pat_match(saying[1:], key[1:])
    return False

In [ ]:
def get_response(saying, rules):

    for key, value in rules.items():
        # 第一步： 得到 pattern = [('?x', 'hello'), ('?y','good')]
        pattern = pat_match(saying.split(),key.split())
        #print("pattern:", pattern)
        if not pattern:
            continue
            
        # 第二步： 将pattern转为字典  pattern = {'?x':'hello', '?y':'good'}
        pattern = pat_to_dict(pattern)
        # 第三步： 获取key对应的一个回答
        answer = value[0]
        # 第四步： 将此回答中的'?x' 用 上面的 pattern 替代
        answer = subsitite(answer.split(),pattern)
        print("B saying :" + ' '.join(answer))
        return
    print("B saying :Sorry, I don't know what you said")

In [ ]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?']
}

In [ ]:
number = 3
for i in range(number):
    saying = input("A saying :")
    get_response(saying, rule_responses)

## 中文版

In [41]:
import jieba

In [42]:
def Ch2zn(saying):
    saying = jieba.lcut(saying)
    while True:
        length = len(saying)
        try:
            index = saying.index('?')
        except:
            break
        else:
            if index == (length - 1):
                break
            if saying[index+1] == '*':
                saying[index] = '?*' + saying[index+2]
                del saying[index+2]
                del saying[index+1]
            else:
                saying[index] = '?' + saying[index+1]
                del saying[index+1]
    return saying

In [43]:
def get_response_Chinese(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """

    for key, value in rules.items():

        # 第一步： 得到 pattern = [('?x', 'hello'), ('?y','good')]
        pattern = pat_match(Ch2zn(saying),Ch2zn(key))

        if not pattern:
            continue
            
        # 第二步： 将pattern转为字典  pattern = {'?x':'hello', '?y':'good'}
        pattern = pat_to_dict(pattern)
        # 第三步： 获取key对应的一个回答
        answer = value[0]
        # 第四步： 将此回答中的'?x' 用 上面的 pattern 替代
        answer = subsitite(Ch2zn(answer),pattern)
        print("B saying :" + ' '.join(answer))
        return
    print("B saying :Sorry, I don't know what you said")

In [46]:
# -*- coding: utf-8 -*-
rule_responses = {
    '你好，我叫?*X, 很高兴认识你': ["?X你好，我也很高兴认识你"], 
    '我喜欢?*y': ['你为什么喜欢?*y？']
}
saying = input("A saying :")
get_response_Chinese(saying, rule_responses)

A saying :你好，我叫MOS, 很高兴认识你
B saying : 你好 ， 我 也 很 高兴 认识 你
